<a href="https://colab.research.google.com/github/VictorManuelOrt/Biometric/blob/main/AlterRanking_Biometrics_DATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install opencv-python numpy


In [2]:
!pip install onnx onnxruntime pydantic==1.10.16 huggingface-hub pyyaml
!pip install --no-deps open-iris

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.1/151.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.4/160.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.12.3
    Uninstalling pydantic-2.12.3:
      Successfully uninstalled pydantic-2.12.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langsmith 0.6.4 requires pydantic<3,>=2, but you have pydantic 1.10.16 which is incompatible.
google-genai 1.55.0 requires pydantic<3.0.0,>=2.9.0, but you have pydantic 1.10.16 which is i

In [39]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("olankadhim/multimodal-biometric-dataset-mulb")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'multimodal-biometric-dataset-mulb' dataset.
Path to dataset files: /kaggle/input/multimodal-biometric-dataset-mulb


In [40]:
import iris
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import os
from PIL import Image
import matplotlib.pyplot as plt
import os
import math
import pandas as pd

In [41]:
from google.colab.patches import cv2_imshow

In [42]:
path

'/kaggle/input/multimodal-biometric-dataset-mulb'

In [43]:
def gabor_kernel(theta, phi, sigma):
    """
    Construye un kernel de Gabor complejo:

    g(x,y) = exp(-(x^2 + y^2)/sigma^2) *
             exp(2*pi*i*theta*(x*cos(phi) + y*sin(phi)))

    El tamaño del kernel se fija como +/- 3*sigma.
    """

    # Tamaño del kernel (impar)
    half = int(np.ceil(3 * sigma))
    K = 2 * half + 1

    # Sistema de coordenadas centrado en (0,0)
    x = np.arange(-half, half + 1)
    y = np.arange(-half, half + 1)
    X, Y = np.meshgrid(x, y)

    # Proyección direccional
    s = X * np.cos(phi) + Y * np.sin(phi)

    # Gaussiana (ventana suave)
    gaussian = np.exp(-(X**2 + Y**2) / sigma**2)

    # Onda compleja (frecuencia + orientación)
    wave = np.exp(2j * np.pi * theta * s)

    # Kernel de Gabor complejo
    kernel = gaussian * wave

    return kernel

def apply_gabor(image, kernel):
    """
    Aplica un filtro de Gabor complejo a una imagen.
    Retorna la magnitud de la respuesta.
    """

    # Asegurar tipo float
    image = image.astype(np.float64)

    # Separar partes real e imaginaria del kernel
    real_k = np.real(kernel)
    imag_k = np.imag(kernel)

    # Convolución (modo 'same' mantiene tamaño)
    real_resp = cv.filter2D(image, cv.CV_64F, real_k)
    imag_resp = cv.filter2D(image, cv.CV_64F, imag_k)

    # Magnitud (energía local)
    magnitude = np.sqrt(real_resp**2 + imag_resp**2)

    return magnitude

In [44]:
def distancia_euclidiana(dic1, dic2, clave1, clave2):
    v1 = np.array(dic1[clave1])
    v2 = np.array(dic2[clave2])
    return np.linalg.norm(v1 - v2)


In [45]:
# -----------------------------
# Parámetros fijos
# -----------------------------
sigma = 80.0
kernel_size = int(2 * np.ceil(3 * sigma) + 1)

# 3 frecuencias espaciales
thetas = [0.02, 0.04, 0.08, 0.16, 0.32]

# 4 orientaciones: k*pi/4
phis = [k * np.pi / 4 for k in range(4)]


80.0

In [47]:
#lo siguiente seria juntar todo en uns funcion y  solo ajustar
#en una celda parametrso del filstro de gabor
#nombre de imagen a sacar caracteristicas
#guardar valores en vectores para hacer comparaciones


#umbral de pertenencia en terminos con juegos?

In [48]:
Features_Dictionary = {}

In [49]:
def Add_Features_Dictionary(name_image):
    """
    Genera un vector
    """

    name_img=name_image
    img_pixels = cv.imread('/kaggle/input/multimodal-biometric-dataset-mulb/MULB dataset/iris dataset/'+name_img, cv.IMREAD_GRAYSCALE)
    if img_pixels is None:
      return

    iris_pipeline = iris.IRISPipeline(env=iris.IRISPipeline.DEBUGGING_ENVIRONMENT)

    output = iris_pipeline(iris.IRImage(img_data=img_pixels, image_id="image_id", eye_side="right"))
    iris_visualizer = iris.visualisation.IRISVisualizer()


    out = iris_visualizer.plot_normalized_iris(normalized_iris=iris_pipeline.call_trace['normalization'],)

    plt.savefig("iris_normalized.png", dpi=300, bbox_inches="tight")
    plt.close()

    Features_Vector = []
    # Leer imagen en escala de grises
    img = cv.imread("iris_normalized.png", cv.IMREAD_GRAYSCALE)
    img = img.astype(np.float64) / 255.0
    # Visualización
    fig, axes = plt.subplots(
        len(thetas),
        len(phis),
        figsize=(12, 8),
        sharex=True,
        sharey=True
    )
    for i, theta in enumerate(thetas):
        for j, phi in enumerate(phis):
            # Construir kernel
            kernel = gabor_kernel(theta, phi, sigma)
            # Aplicar filtro
            response = apply_gabor(img, kernel)
            # Normalizar respuesta
            response_norm = response / (response.max() + 1e-8)
            ax = axes[i, j]
            ax.imshow(response_norm, cmap='gray')
            ax.axis('off')
            # Títulos solo en bordes (más limpio)
            ax.set_title(rf"$\theta = {theta},\ \varphi = {j}\pi/4,\mu_c = {round(response.mean(),3)},\sigma_c = {round(response.std(),3)} $", fontsize=9)
            Features_Vector.append(response.mean())
            Features_Vector.append(response.std())
    plt.suptitle(
        rf"Banco de filtros de Gabor (3 frecuencias × 4 orientaciones){name_img}",
        fontsize=14
    )

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.savefig(name_img.replace("/", "_"))
    plt.close()

    # Guardamos las caracteristicas en el diccionario
    Features_Dictionary[name_image] = Features_Vector



In [50]:
mulb_dataset_path = os.path.join(path, 'MULB dataset')
hand_or_iris="iris dataset"
hand_dataset_path = os.path.join(mulb_dataset_path, hand_or_iris)
!ls -F "{hand_dataset_path}"
!ls -F "{os.path.join(hand_dataset_path, 'DO')}"

A/   AL/  AX/  BI/  BU/  CF/  CR/  DC/	DO/  E/   EL/  EX/  FI/  FU/  GF/  S/
AA/  AM/  AY/  BJ/  BV/  CG/  CS/  DD/	DP/  EA/  EM/  EY/  FJ/  FV/  H/   T/
AB/  AN/  AZ/  BK/  BW/  CH/  CT/  DE/	DQ/  EB/  EN/  EZ/  FK/  FW/  I/   U/
AC/  AO/  B/   BL/  BX/  CI/  CU/  DF/	DR/  EC/  EO/  F/   FL/  FX/  J/   V/
AD/  AP/  BA/  BM/  BY/  CJ/  CV/  DG/	DS/  ED/  EP/  FA/  FM/  FY/  K/   W/
AE/  AQ/  BB/  BN/  BZ/  CK/  CW/  DH/	DT/  EE/  EQ/  FB/  FN/  FZ/  L/   X/
AF/  AR/  BC/  BO/  C/	 CL/  CX/  DI/	DU/  EF/  ER/  FC/  FO/  G/   M/   Y/
AG/  AS/  BD/  BP/  CA/  CM/  CY/  DJ/	DV/  EG/  ES/  FD/  FP/  GA/  N/   Z/
AH/  AT/  BE/  BQ/  CB/  CN/  CZ/  DK/	DW/  EH/  ET/  FE/  FQ/  GB/  O/
AI/  AU/  BF/  BR/  CC/  CO/  D/   DL/	DX/  EI/  EU/  FF/  FR/  GC/  P/
AJ/  AV/  BG/  BS/  CD/  CP/  DA/  DM/	DY/  EJ/  EV/  FG/  FS/  GD/  Q/
AK/  AW/  BH/  BT/  CE/  CQ/  DB/  DN/	DZ/  EK/  EW/  FH/  FT/  GE/  R/
10.jpg	12.jpg	14.jpg	16.jpg	18.jpg	1.jpg	2.jpg  4.jpg  6.jpg  8.jpg
11.jpg	13.jpg	15.jpg	17.jpg	1

In [51]:
name_img="CF/5.jpg"
Add_Features_Dictionary(name_img)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


iris_semseg_upp_scse_mobilenetv2.onnx:   0%|          | 0.00/56.1M [00:00<?, ?B/s]

In [52]:
Features_Dictionary

{'CF/5.jpg': [np.float64(122.53316252618777),
  np.float64(108.79036215548216),
  np.float64(53.393433852289355),
  np.float64(39.640002823419124),
  np.float64(220.41370248083163),
  np.float64(162.81384867336527),
  np.float64(45.42041529801966),
  np.float64(32.20305993878486),
  np.float64(67.3798248923344),
  np.float64(86.64128884412476),
  np.float64(22.075934891546524),
  np.float64(15.189717124100266),
  np.float64(159.89002913508097),
  np.float64(150.54514273516335),
  np.float64(28.65893252655828),
  np.float64(28.67618954241838),
  np.float64(55.35437590371945),
  np.float64(101.85062238046693),
  np.float64(7.046203395261673),
  np.float64(4.440921493756674),
  np.float64(169.16250070613444),
  np.float64(158.68568160374917),
  np.float64(8.059789133015277),
  np.float64(5.82191359242555),
  np.float64(20.89881440411947),
  np.float64(40.10455974991622),
  np.float64(1.891080181174627),
  np.float64(1.5349690244514789),
  np.float64(74.25071783747097),
  np.float64(70.502

In [53]:
Features_Dictionary={}

In [54]:
claves=['R/10.jpg',
 'R/11.jpg',
 'S/9.jpg',
 'S/8.jpg',
 'EL/10.jpg',
 'EL/4.jpg',
 'AL/6.jpg',
 'AL/5.jpg',
 'BR/5.jpg',
 'BR/11.jpg',
 'A/11.jpg',
 'A/9.jpg',
 'BI/5.jpg',
 'BI/16.jpg',
 'CF/10.jpg',
 'CF/6.jpg']

In [55]:
for name_img in claves:
  img_pixels = cv.imread('/kaggle/input/multimodal-biometric-dataset-mulb/MULB dataset/iris dataset/'+name_img, cv.IMREAD_GRAYSCALE)
  if img_pixels is None:
      print(name_img)


In [56]:
for name_img in claves:
  Add_Features_Dictionary(name_img)

In [57]:
claves = list(Features_Dictionary.keys())
claves

['R/10.jpg',
 'R/11.jpg',
 'S/9.jpg',
 'S/8.jpg',
 'EL/10.jpg',
 'EL/4.jpg',
 'AL/6.jpg',
 'AL/5.jpg',
 'BR/5.jpg',
 'BR/11.jpg',
 'A/11.jpg',
 'A/9.jpg',
 'BI/5.jpg',
 'BI/16.jpg',
 'CF/10.jpg',
 'CF/6.jpg']

In [58]:
Features_Dictionary

{'R/10.jpg': [np.float64(115.99695189862149),
  np.float64(99.97223455699674),
  np.float64(44.73468144416728),
  np.float64(32.718757658071134),
  np.float64(212.95096766344713),
  np.float64(155.6768205856783),
  np.float64(48.97808540672968),
  np.float64(39.43133859445016),
  np.float64(79.02961531797452),
  np.float64(75.64716799257563),
  np.float64(22.228866086865164),
  np.float64(20.679702416677017),
  np.float64(168.40695013806015),
  np.float64(149.02442917378067),
  np.float64(24.24160445673644),
  np.float64(19.107198647439407),
  np.float64(57.814508157383756),
  np.float64(101.5165346621429),
  np.float64(8.548251670867257),
  np.float64(6.356838036110703),
  np.float64(173.6218667714106),
  np.float64(160.85623175021365),
  np.float64(9.688633489274721),
  np.float64(8.03846271505584),
  np.float64(23.82714577578514),
  np.float64(41.2519235524896),
  np.float64(3.1207886360278776),
  np.float64(2.55418818911134),
  np.float64(73.0491005462681),
  np.float64(68.72364837

In [59]:
Test_Dictionary={}

In [60]:
def Add_Test_Dictionary(name_image):
    """
    Genera un vector
    """

    name_img=name_image
    img_pixels = cv.imread('/kaggle/input/multimodal-biometric-dataset-mulb/MULB dataset/iris dataset/'+name_img, cv.IMREAD_GRAYSCALE)
    if img_pixels is None:
      return

    iris_pipeline = iris.IRISPipeline(env=iris.IRISPipeline.DEBUGGING_ENVIRONMENT)

    output = iris_pipeline(iris.IRImage(img_data=img_pixels, image_id="image_id", eye_side="right"))
    iris_visualizer = iris.visualisation.IRISVisualizer()


    out = iris_visualizer.plot_normalized_iris(normalized_iris=iris_pipeline.call_trace['normalization'],)

    plt.savefig("iris_normalized.png", dpi=300, bbox_inches="tight")
    plt.close()

    Features_Vector = []
    # Leer imagen en escala de grises
    img = cv.imread("iris_normalized.png", cv.IMREAD_GRAYSCALE)
    img = img.astype(np.float64) / 255.0
    # Visualización
    fig, axes = plt.subplots(
        len(thetas),
        len(phis),
        figsize=(12, 8),
        sharex=True,
        sharey=True
    )
    for i, theta in enumerate(thetas):
        for j, phi in enumerate(phis):
            # Construir kernel
            kernel = gabor_kernel(theta, phi, sigma)
            # Aplicar filtro
            response = apply_gabor(img, kernel)
            # Normalizar respuesta
            response_norm = response / (response.max() + 1e-8)
            ax = axes[i, j]
            ax.imshow(response_norm, cmap='gray')
            ax.axis('off')
            # Títulos solo en bordes (más limpio)
            ax.set_title(rf"$\theta = {theta},\ \varphi = {j}\pi/4,\mu_c = {round(response.mean(),3)},\sigma_c = {round(response.std(),3)} $", fontsize=9)
            Features_Vector.append(response.mean())
            Features_Vector.append(response.std())
    plt.suptitle(
        rf"Banco de filtros de Gabor (3 frecuencias × 4 orientaciones){name_img}",
        fontsize=14
    )

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.savefig(name_img.replace("/", "_"))
    plt.close()

    # Guardamos las caracteristicas en el diccionario
    Test_Dictionary[name_image] = Features_Vector


In [61]:

claves_test=['R/15.jpg',
 'S/14.jpg',
 'EL/6.jpg',
 'AL/7.jpg',
 'BR/10.jpg', 'AW/10.jpg']


In [ ]:
#Add_Test_Dictionary('AW/10.jpg')

In [62]:
for name_img in claves_test:
  Add_Test_Dictionary(name_img)

In [63]:
Test_Dictionary

{'R/15.jpg': [np.float64(116.11988130581146),
  np.float64(102.4477123340748),
  np.float64(54.90712334127434),
  np.float64(40.31346441934172),
  np.float64(236.42324647805415),
  np.float64(156.80884732423448),
  np.float64(58.90629412202324),
  np.float64(60.339171548090405),
  np.float64(74.47496843443459),
  np.float64(81.54390839157628),
  np.float64(24.77855234751191),
  np.float64(17.868986562948145),
  np.float64(180.13545080211813),
  np.float64(155.75442480589422),
  np.float64(24.7710864650757),
  np.float64(19.814246587458207),
  np.float64(55.02178351763667),
  np.float64(100.71087260021505),
  np.float64(10.321933696776256),
  np.float64(6.6599136985386),
  np.float64(170.0195973171411),
  np.float64(161.16974493119952),
  np.float64(8.253998400733261),
  np.float64(4.846649309157055),
  np.float64(21.873535450165775),
  np.float64(40.02639270717725),
  np.float64(2.4838740254633302),
  np.float64(1.6101285328696844),
  np.float64(71.16102114223631),
  np.float64(67.9039

In [64]:

df = pd.DataFrame(index=claves, columns=claves_test)

for i in claves:
    for j in claves_test:
        df.loc[i, j] = distancia_euclidiana(Features_Dictionary, Test_Dictionary, i, j)

for col in df.columns:
    orden = df[col].sort_values(ascending=True)
    print(f"Columna {col}:")
    print(orden)
    print()
coincidencias = {}

for col in df.columns:
    # ordenamos la columna ascendente y obtenemos los índices
    indices_ordenados = df[col].sort_values(ascending=True).index

    # tomamos los dos primeros caracteres de la columna
    prefix_col = col[:2]

    # tomamos los dos primeros índices de la columna ordenada
    primeros_indices = indices_ordenados[:3]

    # contamos cuántos coinciden en los dos primeros caracteres
    count = sum(1 for idx in primeros_indices if idx[:2] == prefix_col)

    coincidencias[col] = count

print(coincidencias)



Columna R/15.jpg:
R/10.jpg     39.502666
CF/6.jpg      42.84243
R/11.jpg     43.065346
S/9.jpg      46.907134
AL/5.jpg     58.515529
CF/10.jpg    59.819938
S/8.jpg       63.81314
AL/6.jpg     65.665495
BR/11.jpg    73.229322
BR/5.jpg     74.322545
BI/5.jpg     82.030561
EL/10.jpg    85.678833
A/11.jpg     85.963255
A/9.jpg       86.45104
BI/16.jpg    88.346406
EL/4.jpg     89.393574
Name: R/15.jpg, dtype: object

Columna S/14.jpg:
S/8.jpg        38.74478
S/9.jpg       47.596975
R/11.jpg      60.103052
CF/6.jpg      61.329515
AL/5.jpg      61.937806
A/9.jpg       66.717606
A/11.jpg      66.775766
BI/5.jpg      69.475289
BI/16.jpg      71.92925
AL/6.jpg      73.262096
R/10.jpg      76.578746
BR/11.jpg     81.358757
BR/5.jpg       81.68135
CF/10.jpg     86.143526
EL/10.jpg     94.224688
EL/4.jpg     112.780906
Name: S/14.jpg, dtype: object

Columna EL/6.jpg:
EL/4.jpg      21.569193
CF/10.jpg     56.650404
BR/5.jpg      68.460763
BR/11.jpg     70.342823
EL/10.jpg     71.578311
R/10.jpg    

In [65]:
valores_dos = sum(1 for v in coincidencias.values() if v >= 2)
print(valores_dos)

4


In [66]:
len(coincidencias)

6



```
# Lo siguiente es el ranking con disancia educlidiana
```



Referencias:
- Tahmasebi, A., & Pourghassem, H. (2017). Robust intra-class distance-based approach for multimodal biometric game theory-based rank-level fusion of ear, palmprint and signature. Iranian Journal of Science and Technology, Transactions of Electrical Engineering, 41(1), 51-64.
- Worldcoin AI. (2023). IRIS: Iris recognition inference system of the Worldcoin project [Computer software]. https://github.com/worldcoin/open-iris